In [1]:
from google.colab import files
files.upload()  # upload the kaggle.json you downloaded


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"navyagn","key":"3f7f2f2cc0750574da55f7d5e1632479"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json



In [ ]:
!kaggle datasets download -d vangap/indian-supreme-court-judgments


Dataset URL: https://www.kaggle.com/datasets/vangap/indian-supreme-court-judgments
License(s): CC-BY-NC-SA-4.0
indian-supreme-court-judgments.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip indian-supreme-court-judgments.zip -d /content/indian_judgments

Archive:  indian-supreme-court-judgments.zip
replace /content/indian_judgments/judgments.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import pandas as pd
import os
import shutil

base_dir = '/content/indian_judgments'  # location of extracted dataset
csv_path = os.path.join(base_dir, 'judgments.csv')
pdf_dir = os.path.join(base_dir, 'pdfs')

# ======== STEP 1: Load dataset ========
judgments_df = pd.read_csv(csv_path)

# Keep only English judgments (where language is NaN)
judgments_df = judgments_df[judgments_df['language'].isna()].reset_index(drop=True)

# ======== STEP 2: Get list of PDFs ========
pdf_files = os.listdir(pdf_dir)
print("Total PDFs available:", len(pdf_files))

# ======== STEP 3: Filter judgments that have matching PDF ========
judgments_df['pdf_exists'] = judgments_df['diary_no'].astype(str).apply(
    lambda x: any(x in fname for fname in pdf_files)
)
judgments_df = judgments_df[judgments_df['pdf_exists']].reset_index(drop=True)
print("Judgments with PDFs:", len(judgments_df))

# ======== STEP 4: Take 2000 random samples ========
reduced_df = judgments_df.sample(n=2000, random_state=42).reset_index(drop=True)

# ======== STEP 5: Add full PDF path column ========
reduced_df['pdf_path'] = reduced_df['diary_no'].astype(str).apply(
    lambda x: next((os.path.join(pdf_dir, f) for f in pdf_files if x in f), None)
)

# ======== STEP 6: Save reduced CSV ========
reduced_csv_path = '/content/reduced_judgments.csv'
reduced_df.to_csv(reduced_csv_path, index=False)
print("Reduced CSV saved at:", reduced_csv_path)

# ======== STEP 7: Copy corresponding PDFs to a new folder ========
reduced_pdf_dir = '/content/reduced_pdfs'
os.makedirs(reduced_pdf_dir, exist_ok=True)

for pdf_path in reduced_df['pdf_path']:
    if pdf_path and os.path.exists(pdf_path):
        shutil.copy(pdf_path, reduced_pdf_dir)

print("Reduced PDFs copied to:", reduced_pdf_dir)
print("Number of PDFs in reduced folder:", len(os.listdir(reduced_pdf_dir)))


Total PDFs available: 48294
Judgments with PDFs: 46212
Reduced CSV saved at: /content/reduced_judgments.csv
Reduced PDFs copied to: /content/reduced_pdfs
Number of PDFs in reduced folder: 1944


In [ ]:
reduced_csv_path = '/content/reduced_judgments.csv'
reduced_pdf_dir = '/content/reduced_pdfs'

# Load CSV
df = pd.read_csv(reduced_csv_path)
# Get list of PDFs in the folder
pdf_files = os.listdir(reduced_pdf_dir)

# Function to check if PDF exists for each diary_no
def check_pdf_exists(row):
    diary_no = str(row['diary_no'])
    for f in pdf_files:
        if f.startswith(diary_no):
            return True
    return False

# Apply function to each row
df['pdf_exists'] = df.apply(check_pdf_exists, axis=1)

# Summary
print("Judgements with matching PDFs:", df['pdf_exists'].sum())
print("Judgements without matching PDFs:", len(df) - df['pdf_exists'].sum())

Judgements with matching PDFs: 1937
Judgements without matching PDFs: 63


In [ ]:
#Keep only rows with matching PDFs
df = df[df['pdf_exists']].reset_index(drop=True)

# Keep only relevant columns
columns_to_keep =  ['diary_no', 'case_no', 'judgment_dates', 'pdf_exists', 'pdf_path']
df = df[columns_to_keep]

# Save cleaned CSV
clean_csv_path = '/content/reduced_judgements_clean.csv'
df.to_csv(clean_csv_path, index=False)

print("Shape of cleaned dataset:", df.shape)
print("Columns after cleaning:", df.columns.tolist())
df.head()


Shape of cleaned dataset: (1937, 5)
Columns after cleaning: ['diary_no', 'case_no', 'judgment_dates', 'pdf_exists', 'pdf_path']


,diary_no,case_no,judgment_dates,pdf_exists,pdf_path
0,80384-1991,- 0,10-03-1992,True,/content/indian_judgments/pdfs/80384-1991___jo...
1,18732-1998,Crl.A. No.-001258-001263 - 1999,26-11-1999,True,/content/indian_judgments/pdfs/18732-1998___jo...
2,7011-2000,C.A. No.-002551-002551 - 2001,20-02-2008,True,/content/indian_judgments/pdfs/7011-2000___jon...
3,25859-2004,C.A. No.-000584-000584 - 2008,22-01-2008,True,/content/indian_judgments/pdfs/25859-2004___jo...
4,9232-2003,C.A. No.-004227-004227 - 2003,03-09-2009,True,/content/indian_judgments/pdfs/9232-2003___jon...
